In [1]:
from allennlp.predictors.predictor import Predictor
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz"
predictor = Predictor.from_path(model_url)

error loading _jsonnet (this is expected on Windows), treating C:\Users\TANVIK~1\AppData\Local\Temp\tmpvct0ivcr\config.json as plain json
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import itertools
import re
import nltk
import string
import pandas as pd
import numpy as np
import rouge
ROUGE = rouge.Rouge(metrics=['rouge-n'],
                           max_n=1)
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import ngrams
from nltk.corpus import stopwords
from collections import OrderedDict

In [3]:
df = pd.read_csv('datasets/wikihowAll.csv')

In [4]:
df = df.drop(['title'], axis = 1)
df = df.head(100)
df

,headline,text
0,"\nKeep related supplies in the same area.,\nMa...","If you're a photographer, keep all the necess..."
1,\nCreate a sketch in the NeoPopRealist manner ...,See the image for how this drawing develops s...
2,"\nGet a bachelor’s degree.,\nEnroll in a studi...",It is possible to become a VFX artist without...
3,\nStart with some experience or interest in ar...,The best art investors do their research on t...
4,"\nKeep your reference materials, sketches, art...","As you start planning for a project or work, ..."
...,...,...
95,"\nWatch out for a shifting saddle.,\nNotice if...",If you feel that you are having to lean to on...
96,"\nBuild a trustworthy bond with your piggy.,\n...",Having a stable relationship with an animal m...
97,"\nLet your guinea pig out of its cage.,\nMake ...",";\n,,,,, Make sure every time you do this you ..."
98,\nCondition your guinea pig to the clicker or ...,"To do this, put your guinea pig in a safe are..."


In [5]:
df['headline'] = df['headline'].apply(lambda x: x.replace(',\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace(',\n', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n,', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n,', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n\n', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n\n\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n\n\n', ' '))
df['headline'] = df['headline'].apply(lambda x: x.replace('\n', ' '))
df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))

In [6]:
df

,headline,text
0,Keep related supplies in the same area. Make ...,"If you're a photographer, keep all the necess..."
1,Create a sketch in the NeoPopRealist manner o...,See the image for how this drawing develops s...
2,Get a bachelor’s degree. Enroll in a studio-b...,It is possible to become a VFX artist without...
3,Start with some experience or interest in art...,The best art investors do their research on t...
4,"Keep your reference materials, sketches, arti...","As you start planning for a project or work, ..."
...,...,...
95,Watch out for a shifting saddle. Notice if yo...,If you feel that you are having to lean to on...
96,Build a trustworthy bond with your piggy. Res...,Having a stable relationship with an animal m...
97,Let your guinea pig out of its cage. Make sur...,"; ,,,, Make sure every time you do this you sa..."
98,Condition your guinea pig to the clicker or v...,"To do this, put your guinea pig in a safe are..."


In [7]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def sentence_segment(self, doc, lower):
        sentences = []
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sents = re.split(pattern, doc)
        sents = [sentence.split() for sentence in sents if any(char.isalpha() for char in sentence)]
        
        for sent in sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if lower is True:
                    selected_words.append(token.lower())
                else:
                    selected_words.append(token)
            sentences.append(selected_words)
        return sentences
        
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                window_size=4, lower=False):
        """Main function to analyze text"""
        
        # Filter sentences
        sentences = self.sentence_segment(text, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [8]:
def summarize_text(dataframe):
    # Filling in Nan values
    dataframe['text'] = dataframe['text'].fillna('This abstract does not exist')

    # Generating best summary for each abstract
    summaries = []
    scores = []
    resolved = []
    for i in range(len(dataframe)):
        abstract = dataframe.loc[i, 'text']
        
        # Allen Anaphora Resolution
        prediction = predictor.predict(document=abstract)  # get prediction
        abstract = predictor.coref_resolved(abstract)  # resolved text
        resolved.append(abstract)
        
        headline = dataframe.loc[i, 'headline']
        
        # TextRank
        textRank = TextRank4Keyword()
        textRank.analyze(abstract)
        text_rank_scores = textRank.node_weight

        # Sentence Tokenization
        pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
        sentences = re.split(pattern, abstract)
        sentences = [sentence for sentence in sentences if any(char.isalpha() for char in sentence)]
        headline_sentences = re.split(pattern, headline)
        headline_sentences = [sentence for sentence in headline_sentences if any(char.isalpha() for char in sentence)]
        n_sentences = len(headline_sentences)

        # Sentence scoring based on textrank values
        scored_sentences = []
        for index, sentence in enumerate(sentences):
            words = sentence.split(' ')
            score = sum(text_rank_scores[word] if word in text_rank_scores else 0.0 for word in words)
            scored_sentences.append((sentence, score, index))
        scored_sentences.sort(key=lambda x: x[1], reverse=True)

        # Top n sentences are taken as summary
        top_sentences = []
        for sentence, score, index in scored_sentences[:n_sentences]:
            top_sentences.append((sentence, index))
        top_sentences.sort(key=lambda x: x[1])
        top_sentences = [sentence for sentence, index in top_sentences]
        summary = ' '.join(top_sentences)
        summaries.append(summary)
        
        # Rouge-1 score calculation
        score = ROUGE.get_scores(summary, headline)
        p = score["rouge-1"]["p"]
        r = score["rouge-1"]["r"]
        f1 = score["rouge-1"]["f"]
        scores.append([p, r, f1])

    dataframe['resolved text'] = resolved
    dataframe['summary'] = summaries
    dataframe['rouge-1 score'] = scores
    
    return dataframe

In [11]:
output = summarize_text(df.head(1))

C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_14968\3420199793.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['text'] = dataframe['text'].fillna('This abstract does not exist')
C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_14968\3420199793.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['resolved text'] = resolved
C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_14968\3420199793.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [12]:
output

,headline,text,resolved text,summary,rouge-1 score
0,Keep related supplies in the same area. Make ...,"If you're a photographer, keep all the necess...","If you're a photographer, keep all the necess...","Infrequent or secondary supplies area, tucked ...","[0.26495726495726496, 0.36470588235294116, 0.3..."


In [13]:
print(output['text'][0])

 If you're a photographer, keep all the necessary lens, cords, and batteries in the same quadrant of your home or studio. Paints should be kept with brushes, cleaner, and canvas, print supplies should be by the ink, etc. Make broader groups and areas for your supplies to make finding them easier, limiting your search to a much smaller area. Some ideas include:  Essential supplies area -- the things you use every day. Inspiration and reference area. Dedicated work area . Infrequent or secondary supplies area, tucked out of the way.;  This doesn't mean cleaning the entire studio, it just means keeping the area immediately around the desk, easel, pottery wheel, etc. clean each night. Discard trash or unnecessary materials and wipe down dirty surfaces. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.  Even if the rest of your studio is a bit disorganized, an organized workspace will help you

In [14]:
print(output['summary'][0])

Infrequent or secondary supplies area, tucked out of the way.;  This doesn't mean cleaning the entire studio, This just means keeping the area immediately around the desk, easel, pottery wheel, etc. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.  Even if the rest of your studio is a bit disorganized, an organized workspace will help you get down to business every time you want to make art. Plastic restaurant squirt bottles for paint, pigment, etc., Simply string up the wires across a wall or along the ceiling and use the wires to hold essential papers that you don't want to cut or ruin with tacks or tape. Cheap and easy, this is also a good way to handle papers and ideas you touch regularly or need to pin up and down for inspiration., Shelving is an artist's best friend and is a cheap and easy way to get more room in your studio or art space. The upper reaches of the room are often the

In [15]:
print(output['headline'][0])

 Keep related supplies in the same area. Make an effort to clean a dedicated workspace after every session. Place loose supplies in large, clearly visible containers. Use clotheslines and clips to hang sketches, photos, and reference material. Use every inch of the room for storage, especially vertical space. Use chalkboard paint to make space for drafting ideas right on the walls. Purchase a label maker to make your organization strategy semi-permanent. Make a habit of throwing out old, excess, or useless stuff each month.


In [17]:
p_sum = r_sum = f1_sum = 0
for score in output['rouge-1 score']:
    p_sum += score[0]
    r_sum += score[1]
    f1_sum += score[2]
total = len(output['rouge-1 score'])
avg = [p_sum/total, r_sum/total, f1_sum/total]
print(avg)

[0.26495726495726496, 0.36470588235294116, 0.3069306930693069]


In [20]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
resolved = nlp(output['summary'][0])

NER = []
for X in resolved.ents:
    NER.append((X.text, X.label_))

In [22]:
NER, output['summary'][0]

([('the next day', 'DATE'), ('30 minutes', 'TIME')],
 "Infrequent or secondary supplies area, tucked out of the way.;  This doesn't mean cleaning the entire studio, This just means keeping the area immediately around the desk, easel, pottery wheel, etc. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.  Even if the rest of your studio is a bit disorganized, an organized workspace will help you get down to business every time you want to make art. Plastic restaurant squirt bottles for paint, pigment, etc., Simply string up the wires across a wall or along the ceiling and use the wires to hold essential papers that you don't want to cut or ruin with tacks or tape. Cheap and easy, this is also a good way to handle papers and ideas you touch regularly or need to pin up and down for inspiration., Shelving is an artist's best friend and is a cheap and easy way to get more room in your studio or

In [27]:
weights = np.zeros(len(NER))
for i in range(0,len(weights)):
    weights[i] = np.random.rand()

In [28]:
weights

array([0.62054909, 0.52828858])

In [29]:
num_weights = len(weights)

In [30]:
import numpy
sol_per_pop = 8
pop_size = (sol_per_pop,num_weights) # The population will have sol_per_pop chromosome where each chromosome has num_weights genes.

#Creating the initial population.
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

In [31]:
new_population

array([[-2.41826796,  1.9907412 ],
       [-3.84006928,  2.63599761],
       [-0.81653474,  0.85471756],
       [-2.1724872 , -0.91582156],
       [-2.14773374, -2.59154265],
       [ 0.20894618,  3.74137758],
       [ 1.07838217,  2.88186756],
       [ 1.40077059,  2.94948967]])

In [32]:
def cal_pop_fitness(equation_inputs, pop):
     # Calculating the fitness value of each solution in the current population.
     # The fitness function calculates the sum of products between each input and its corresponding weight.
     fitness = numpy.sum(pop*equation_inputs, axis=1)
     return fitness

In [33]:
def select_mating_pool(pop, fitness, num_parents):

    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.

    parents = numpy.empty((num_parents, pop.shape[1]))

    for parent_num in range(num_parents):

        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))

        max_fitness_idx = max_fitness_idx[0][0]

        parents[parent_num, :] = pop[max_fitness_idx, :]

        fitness[max_fitness_idx] = -99999999999

    return parents

In [34]:
def crossover(parents, offspring_size):
     offspring = numpy.empty(offspring_size)
     # The point at which crossover takes place between two parents. Usually, it is at the center.
     crossover_point = numpy.uint8(offspring_size[1]/2)
 
     for k in range(offspring_size[0]):
         # Index of the first parent to mate.
         parent1_idx = k%parents.shape[0]
         # Index of the second parent to mate.
         parent2_idx = (k+1)%parents.shape[0]
         # The new offspring will have its first half of its genes taken from the first parent.
         offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
         # The new offspring will have its second half of its genes taken from the second parent.
         offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
     return offspring

In [41]:
def mutation(offspring_crossover, length):

    # Mutation changes a single gene in each offspring randomly.

    for idx in range(offspring_crossover.shape[0]):

        # The random value to be added to the gene.

        random_value = numpy.random.uniform(-1.0, 1.0, 1)

        offspring_crossover[idx, length-1] = offspring_crossover[idx, length-1] + random_value

    return offspring_crossover

In [42]:
num_generations = 5

num_parents_mating = 4
for generation in range(num_generations):
     # Measuring the fitness of each chromosome in the population.
     fitness = cal_pop_fitness(weights, new_population)
    # Selecting the best parents in the population for mating.
     parents = select_mating_pool(new_population, fitness, 
                                       num_parents_mating)
 
     # Generating next generation using crossover.
     offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-parents.shape[0], num_weights))
 
     # Adding some variations to the offspring using mutation.
     offspring_mutation = mutation(offspring_crossover, len(weights))# Creating the new population based on the parents and offspring.
     new_population[0:parents.shape[0], :] = parents
     new_population[parents.shape[0]:, :] = offspring_mutation

C:\Users\Tanvi Kandalla\AppData\Local\Temp\ipykernel_14968\3511615979.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  offspring_crossover[idx, length-1] = offspring_crossover[idx, length-1] + random_value


In [43]:
new_population

array([[1.40077059, 3.86470504],
       [1.40077059, 3.47397848],
       [1.40077059, 3.40567017],
       [1.40077059, 2.94948967],
       [1.40077059, 2.83610371],
       [1.40077059, 4.19259805],
       [1.40077059, 3.70706767],
       [1.40077059, 3.49478513]])